# Initial Imports

In [1]:
import os
import sys

# Get the current working directory (where the Jupyter Notebook is being executed)
current_file_dir = os.getcwd()  # This works in Jupyter Notebooks

# Add the subdirectories to sys.path
data_path = os.path.join(current_file_dir, '../data')  # Removed '../' since it's relative to cwd
python_path = os.path.join(current_file_dir, '../python')

sys.path.append(data_path)
sys.path.append(python_path)

# Initial System Dictionary

In [2]:
import numpy as np

# Define the instrument configuration as a system dictionary
system_dict = {
    "components": {
        "parang_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "m3": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * 0, "epsilon": 0},
            "tag": "on-sky",
        },
        "alt_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "lp": {
            "type": "LinearPolarizer",
            "properties": {"theta": 0},
            "tag": "internal",
        },
        "hwp": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0.5, "delta_theta": 0},
            "tag": "internal",
        },
        "image_rotator": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0, "delta_theta": 0},
            "tag": "internal",
        },
        "optics": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * 0, "epsilon": 0},
            "tag": "on-sky",
        },
        "flc": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0, "theta": 0},
            "tag": "internal",
        },
        "dichroic": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0, "theta": 0},
            "tag": "internal",
        },
        "wollaston": {
            "type": "WollastonPrism",
            "properties": {"beam": "o", "transmission_ratio": 1},
            "tag": "internal",
        },
    },
    # Downstream to upstream
    "order": [
        "wollaston", "dichroic", "flc", "optics", "image_rotator", "hwp", "lp", 
        "alt_rot", "m3", "parang_rot"
    ],
    # Upstream to downstream
    # "order": [
    #     "wollaston", "hwp"
    # ]
}

# generate_system_mueller_matrix

In [3]:
# Importing modules from the python subdirectory
from instrument_dictionaries import generate_system_mueller_matrix

system_mm = generate_system_mueller_matrix(system_dict)
print(system_mm.evaluate())

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


# update_system_mueller_matrix

In [4]:
# TODO: Not sure if this is strange with the negative HWP angles

from instrument_dictionaries import update_system_mm

parameter_values = [45, "e"]
system_parameters = [["hwp", "theta"], 
                    ["wollaston", "beam"]
                    ]
# updated_system_mm = update_system_mm(parameter_updates, system_mm)
updated_system_mm = update_system_mm(parameter_values, system_parameters, system_mm)
print(updated_system_mm.evaluate())

[[ 0.5  0.5  0.   0. ]
 [-0.5 -0.5  0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]]


# generate_measurement

In [5]:
from instrument_dictionaries import generate_measurement

generate_measurement(system_mm, s_in = [1, 1, 0, 0])

array([ 1., -1.,  0.,  0.])

# parse_configuration

In [7]:
from instrument_dictionaries import parse_configuration

print(parse_configuration(system_dict))

([0, 0.0, 0, 0, 0, 3.141592653589793, 0, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0, 'o', 1], [['parang_rot', 'pa'], ['m3', 'phi'], ['m3', 'epsilon'], ['alt_rot', 'pa'], ['lp', 'theta'], ['hwp', 'phi'], ['hwp', 'delta_theta'], ['image_rotator', 'phi'], ['image_rotator', 'delta_theta'], ['optics', 'phi'], ['optics', 'epsilon'], ['flc', 'phi'], ['flc', 'theta'], ['dichroic', 'phi'], ['dichroic', 'theta'], ['wollaston', 'beam'], ['wollaston', 'transmission_ratio']])


# log_l